<a href="https://colab.research.google.com/github/Jaat2727/Torrent-Downloader-on-Collab-To-Google-Drive/blob/main/TORRENT_DOWNLOADER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torrentp
!pip install torrentp nest_asyncio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import asyncio
import nest_asyncio
from torrentp import TorrentDownloader
import time
import os
import threading

# Enable nested asyncio (REQUIRED for Colab)
nest_asyncio.apply()

# New Magnet Link (Spider-Man: Far From Home - 2160p BluRay)
MAGNET_LINK = ""
DOWNLOAD_PATH = "/content/drive/MyDrive/MyDownloads"  # Replace with your desired path

# --- Keep-Alive (Prevent Timeout) ---
def keep_alive():
    """
    Simulates user interaction to prevent Colab from timing out.
    Runs in a separate thread.
    """
    while True:
        print("Keep-alive: Simulating activity...")
        time.sleep(60)  # Simulate activity every 60 seconds

# Start the keep-alive thread
keep_alive_thread = threading.Thread(target=keep_alive, daemon=True)
keep_alive_thread.start()
# --- End Keep-Alive ---

async def monitor_download(torrent_file, interval=5):
    """Monitors download progress and prints statistics."""
    while not torrent_file.is_completed():
        stats = torrent_file.get_stats()
        total_size_gb = stats.total_size / (1024**3)
        downloaded_gb = stats.downloaded / (1024**3)
        progress = (stats.downloaded / stats.total_size) * 100 if stats.total_size else 0
        download_speed_mbps = (stats.download_speed / (1024**2)) * 8
        upload_speed_mbps = (stats.upload_speed / (1024**2)) * 8
        num_peers = stats.num_peers

        print(
            f"Progress: {progress:.2f}% | "
            f"Downloaded: {downloaded_gb:.2f} GB / {total_size_gb:.2f} GB | "
            f"Down Speed: {download_speed_mbps:.2f} Mbps | "
            f"Up Speed: {upload_speed_mbps:.2f} Mbps | "
            f"Peers: {num_peers}"
        )
        await asyncio.sleep(interval)

    print("Download Complete!")
    print(torrent_file.get_downloaded_files())

async def main():
    """Downloads a torrent from a magnet link, with monitoring and Colab-specific handling."""
    torrent_file = None
    try:
        torrent_file = TorrentDownloader(MAGNET_LINK, DOWNLOAD_PATH)
        print("Starting download...")

        # Run download and monitoring concurrently
        download_task = asyncio.create_task(torrent_file.start_download())
        monitor_task = asyncio.create_task(monitor_download(torrent_file))

        # Wait for download to complete (or be interrupted)
        await download_task

        # Cancel the monitoring task
        monitor_task.cancel()
        try:
            await monitor_task  # Ensure cancellation is complete
        except asyncio.CancelledError:
            pass

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if torrent_file:
            print("Stopping download...")
            torrent_file.stop_download()

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())